# Model Training, Registration, and Champion Deployment (Unity Catalog)
Train XGBoost on gold features, register the model in Unity Catalog, and set as champion (production alias).

In [ ]:
# Train XGBoost on gold features, register in Unity Catalog, and set as champion
import mlflow
import mlflow.xgboost
import xgboost as xgb
from sklearn.model_selection import train_test_split
import pandas as pd

# Define paths for gold features and Unity Catalog model
gold_path = "/Volumes/your_catalog/your_schema/energy_price_gold_features"
model_uc_path = "models:/your_catalog/your_schema/energy_price_xgb"

mlflow.set_registry_uri("databricks-uc")
mlflow.xgboost.autolog()

gold_pdf = spark.read.format("delta").load(gold_path).toPandas()
feature_cols = [c for c in gold_pdf.columns if c not in ["start_time", "end_time", "price_eur_mwh"]]
X = gold_pdf[feature_cols]
y = gold_pdf["price_eur_mwh"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

with mlflow.start_run(run_name="xgb_train_uc"):
    model = xgb.XGBRegressor()
    model.fit(X_train, y_train)
    mlflow.xgboost.log_model(model, "model", registered_model_name=model_uc_path)
    run_id = mlflow.active_run().info.run_id
    print(f"Model registered to Unity Catalog: {model_uc_path}")

# Set as champion (production alias)
# Note: The following is a placeholder. In Databricks, use the REST API or UI to set the alias.
print("Champion model should be set to the latest version in Unity Catalog (use Databricks UI/API).")